In [1]:
import cv2
import numpy as np
from time import sleep

largura_min = 80
altura_min = 80
offset = 6
pos_linha = 550

# FPS to vídeo
delay = 60
detec = []
carros = 0

def pega_centro(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy

# video source input
cap = cv2.VideoCapture('video.mp4')
subtracao = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame1 = cap.read()        
    tempo = float(1/delay)         
    sleep(tempo)         
    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)         
    blur = cv2.GaussianBlur(grey, (3, 3), 5)          
    img_sub = subtracao.apply(blur)          
    dilat = cv2.dilate(img_sub, np.ones((5, 5)))           

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))           
    dilatada = cv2.morphologyEx(dilat, cv2. MORPH_CLOSE, kernel)          
    dilatada = cv2.morphologyEx(dilatada, cv2. MORPH_CLOSE, kernel)          

    contorno, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)          
    cv2.line(frame1, (25, pos_linha), (1200, pos_linha), (176, 130, 39), 2)            

    for(i, c) in enumerate(contorno):            
        (x, y, w, h) = cv2.boundingRect(c)                
        validar_contorno = (w >= largura_min) and (h >= altura_min)                 
        if not validar_contorno:                 
            continue                      

        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)                    
        centro = pega_centro(x, y, w, h)                    
        detec.append(centro)                     
        cv2.circle(frame1, centro, 4, (0, 0, 255), -1)                    

        for (x, y) in detec:                 
            if (y < (pos_linha + offset)) and (y > (pos_linha-offset)):                         
                carros += 1                          
                cv2.line(frame1, (25, pos_linha), (1200, pos_linha), (0, 127, 255), 3)                         
                detec.remove((x, y))                        
                print("No. of cars detected : " + str(carros))                        


    cv2.putText(frame1, "VEHICLE COUNT : "+str(carros), (320, 70),              cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 4)
    cv2.imshow("Video Original", frame1)
    cv2.imshow(" Detectar ", dilatada)
    if cv2.waitKey(1) == 27:
        break        

    
cv2.destroyAllWindows()
cap.release()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
